In [ ]:
#1 - IMPORTING LIBRARIES
# Pandas and numpy for data manipulation
import pandas as pd
import numpy as np
np.random.seed(42)
pd.set_option('display.max_rows', 600)
 
# Matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib
matplotlib.rcParams['font.size'] = 16
matplotlib.rcParams['figure.figsize'] = (9, 9)
import math

import seaborn as sns

from IPython.core.pylabtools import figsize

# Scipy helper functions
from scipy.stats import percentileofscore
from scipy import stats

#Import scikit learn statistics
from sklearn.metrics import mean_absolute_error, median_absolute_error, f1_score, make_scorer, r2_score

#Import variance inflation factor info from statsmodels
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from statsmodels.regression.linear_model import OLS

#Import scipy for probability distributions
import scipy

#Itertools for combinations
from itertools import combinations

#pdb for debugging
import pdb

# PyMC3 for Bayesian Inference (includes Patsy module)
import pymc3 as pm

In [ ]:
#2 - READ AND CHECK DATA, EXPLORE DISTRIBUTION OF VARIABLES
#Column names don't matter - except 'Erosion_mmkyr' needs to be the erosion data (only column called on by name)

# Read excel table with erosion rates and predictor variables (complete path to excel file location)
df_st = pd.read_excel('/Users/erinseagren/Documents/Papers/CRN_paper/Model_input_vars/best_corr.xlsx')
catchment_name = df_st.iloc[:,0]

#Print column headers (plus first 4 rows) of spreadsheet - check that it looks okay (correct headers, first row seems
#accurate, etc.)
df_st.head()

In [ ]:
#Check shape of dataframe (make sure all data is included, size matches what you expect)
df_st.shape

In [ ]:
#Drop catchment name column (will mess up further code because does not contain numbers) - can change label in code 
#below if it's not called 'Catchment_name'
df_st = df_st.drop(columns=['Catchment_name'])

In [ ]:
#Check for null values in columns
df_st.isnull().values.any()

#Check individual columns with the following - just change the number
#df_st.iloc[:,28].isnull().values.any()

In [ ]:
#Collinearity check with Variance Inflation Factor
#Checks the VIF between variables (1/(1-r2)) between independent variables, VIF > 5 - collinear + need to drop vars/do
#PCA/etc., VIF = 1-5 - somewhat collinear so be careful, VIF = 1 - not collinear

#NB: is based on ordinary least squares

#Make copy of dataframe
df_vif = df_st.copy()

#Drop erosion rate and any other columns related to dependent variables
df_vif = df_vif.drop(columns=['Erosion_mmkyr'])

#Function for calculating VIF and returning dataframe
def variance_inflation_factors(exog_df):
    '''
    Parameters
    ----------
    exog_df : dataframe, (nobs, k_vars)
        design matrix with all explanatory variables, as for example used in
        regression.

    Returns
    -------
    vif : Series
        variance inflation factors
    '''
    #VIF function requires constant (see https://github.com/statsmodels/statsmodels/issues/2376)
    exog_df = add_constant(exog_df)
    vifs = pd.Series(
        [1 / (1. - OLS(exog_df[col].values, 
                       exog_df.loc[:, exog_df.columns != col].values).fit().rsquared) 
         for col in exog_df],
        index=exog_df.columns,
        name='VIF'
    )
    return vifs

variance_inflation_factors(df_vif)

In [ ]:
#Check parameter distributions (erosion rates and all independent variables) through histograms
sns.set_style('darkgrid')
for i, col in enumerate(df_st.columns):
    plt.figure(i,figsize=(5,4))
    #Can turn off KDE estimation with kde=False, can change bin numbers below also
    sns.distplot(df_st[col],bins=20)

In [ ]:
# Calculate percentiles for erosion and predictor variables to further assess normal/non-normal distribution
#Code from: https://github.com/WillKoehrsen/Data-Analysis/blob/master/bayesian_lr/Bayesian%20Linear%20Regression%20Project.ipynb
for i, col in enumerate(df_st.columns):
    df_st['percentile'] = df_st[col].apply(lambda x: percentileofscore(df_st[col], x))
    
    #Calculate maximum values for each variable and convert to integers (for plotting)
    max_value = df_st[col].max()
    maximums = math.ceil(max_value)
    
    #Plot figures
    plt.figure(figsize = (5, 3))
    plt.plot(df_st[col], df_st['percentile'], '.')
    plt.xticks(np.linspace(0, maximums, 5), np.linspace(0, maximums, 5))
    plt.xlabel(df_st.columns[i]); plt.ylabel('Percentile'); plt.title('Variable percentiles')

In [ ]:
#3 - PREP FOR SUMMARY STATS ANALYSIS, MODELLING
#Includes: sampling from erosion rate mean/sigma to account for uncertainty and log transforming all data (base e) 


#NB: Only do the first part (erosion uncertainty sampling) if you want to incorporate uncertainty in erosion rates into
#further modelling, increases time for models to run

In [ ]:
#Erosion uncertainty sampling

#Set number of times to sample from distributions (will slow down Bayesian modelling part a lot if you sample too much):
num_samples = 1000

#Repeat each row in dataframe number of times set above (expands independent variables - they remain the same)
df_st=pd.DataFrame(pd.np.repeat(df_st.values,num_samples,axis=0),columns=df_st.columns)

#Add new column called 'Sampled erosion' and populate with samples from a normal distribution for each erosion rate
#NB: Change names for mean (currently 'Erosion_mmkyr') and sigma (currently 'Erosion_1sigma') if columns are called 
#something else
df_st['Sampled_erosion'] = np.random.normal(df_st['Erosion_mmkyr'],df_st['Erosion_1sigma'])

#Rename 'Sampled_erosion' 'Erosion_mmkyr' (necessary for rest of code) and the original 'Erosion_mmkyr' (mean rates)
#'Mean_erosion_rate'
df_st.rename(columns={"Erosion_mmkyr":"Mean_erosion_rate"},inplace=True)
df_st.rename(columns={"Sampled_erosion":"Erosion_mmkyr"},inplace=True)

df_saved = df_st.copy()

In [ ]:
#Log transform (base e) all data (erosion rates and independent variables)

#Drop any extra columns (e.g. percentile from above, erosion_1sigma, mean_erosion_rate) that you don't want included in
#the regression modelling
df_st = df_st.drop(columns=['percentile','Erosion_1sigma'])

#log transform all remaining data
df_st = np.log(df_st)

#Check resulting dataframe to make sure it makes sense
df_st

In [ ]:
#4 - SUMMARY STATS ANALYSIS, FEATURE SELECTION, RETESTING COLLINEARITY, ETC.
#Compare variables against each other and erosion rates to assess collinearity, determine what seem to be important
#predictor variables, etc.

#Plot pairgrids (scatterplots, kernel density estimations, and histograms of individual variables)
#Create function that calculates Spearman's rho and plots it onto the pairgrid
#Code taken from: https://github.com/WillKoehrsen/Data-Analysis/blob/master/bayesian_lr/Bayesian%20Linear%20Regression%20Project.ipynb
#NB: this takes a while if there are a lot of variables
def corrfunc(x, y, **kws):
    r, _ = stats.spearmanr(x, y)
    ax = plt.gca()
    ax.annotate("p = {:.2f}".format(r),
                xy=(.05, .85), xycoords=ax.transAxes,
               size = 25)

#Initialize pairgrid features like font size, colors, etc.
cmap = sns.cubehelix_palette(light=1, dark = 0.1,
                             hue = 0.9, as_cmap=True)
sns.set_context(font_scale=2)

# Sets up blank pairgrid (add what we want to be included in code below)
g = sns.PairGrid(df_st)

# Set upper triangle (right corner) as scatterplots and plot Spearman's rho from corrfunc above
g.map_upper(plt.scatter, s=10, color = 'purple')
g.map_upper(corrfunc);

# Create histograms on the diagonal (so variable v same variable)
g.map_diag(sns.distplot, kde=False, color = 'purple')

# Set lower triangle as bivariate kernel density estimations
g.map_lower(sns.kdeplot, cmap = cmap)


In [ ]:
#Plot heatmap to visualize correlations bewteen erosion and other variables and between variables

#Choose whether you want to use Spearman's rho or Pearson's r
corr = df_st.corr(method='spearman')
#corr = df_st.corr(method='pearson')

#Plot heatmap with correlations
fig, ax = plt.subplots(figsize=(25,25))  
sns.heatmap(corr, annot=True, linewidths=0.75, fmt='.1f')

In [ ]:
#Create table of Pearson's r, easier to copy than the heatmap, assess correlations all indepedent variables 
#(looking for r > 0.7 as sign of collinearity)

#NB: Function will print correlations between all variables and the one specified - will need to change this by hand
#to assess all independent variables
df_st.corr(method='pearson')['NDVI_90_10'].sort_values()

In [ ]:
#Retest variance inflation factor (after log transforming data) (>5 = severely collinear, 1 = not collinear, 1-5 = 
#moderately collinear - Sheather 2009 'A modern approach to regression with R')
df_vif = df_st.copy()

#Drop erosion rate and any other columns related to dependent variables
df_vif = df_vif.drop(columns=['Erosion_mmkyr'])

#Function for calculating VIF and returning dataframe
def variance_inflation_factors(exog_df):
    '''
    Parameters
    ----------
    exog_df : dataframe, (nobs, k_vars)
        design matrix with all explanatory variables, as for example used in
        regression.

    Returns
    -------
    vif : Series
        variance inflation factors
    '''
    #VIF function requires constant (see https://github.com/statsmodels/statsmodels/issues/2376)
    exog_df = add_constant(exog_df)
    vifs = pd.Series(
        [1 / (1. - OLS(exog_df[col].values, 
                       exog_df.loc[:, exog_df.columns != col].values).fit().rsquared) 
         for col in exog_df],
        index=exog_df.columns,
        name='VIF'
    )
    return vifs

variance_inflation_factors(df_vif)

In [ ]:
#Remove any variables you don't want carried forward (e.g. drainage area), just change to the column name you want
#dropped
df_st = df_st.drop(columns=['Area'])

In [ ]:
#Remove variables with Spearman's rho/Pearson's r absolute value lower than user-specified threshold (need to change
#between Spearman/Pearson by hand below)
#YOU MUST RUN THIS - RENAMES DATAFRAME DF - IF YOU DON'T WANT TO REMOVE VARIABLES, SET THE CUTOFF AT 0
def remove_vars(df_st,threshold):
    
    #Create table that records which each independent variables Spearman's rho value (target is erosion)
    correlated = df_st.corr(method='spearman').abs()['Erosion_mmkyr'].sort_values(ascending=False)
    #correlated = df_st.corr(method='pearson').abs()['Erosion_mmkyr'].sort_values(ascending=False)
    
    #Remove variables with rho value lower than specified threshold
    correlated = correlated[correlated >= threshold]
    
    #Remove variables from dataframe based on correlated index (greater than or equal to threshold value)
    df = df_st.loc[:, correlated.index]

    return df

#Run remove_vars function (set threshold - min absolute rho to keep variables)
threshold = 0
df = remove_vars(df_st,threshold)

#Check updated dataframe - should have removed all variables with rho less than threshold (check box above for which)
df

In [ ]:
#5 - SET UP BAYESIAN MODEL

#Create function to run model (user specifies input formula (e.g. y~x), distribution family, observed data, and number
#of draws and tuning size), function returns pm.summary, traceplots, and the independent variables used
#formula = defined below, family = Normal or StudentT, observed data = df, draws = total # of samples, tune = burn in,
#priors = True or False (true = user-specified priors, false = default priors), my_priors = user-set priors (not 
#required)
def run_model(formula,dist_family,data,draws,tune,prior,my_priors=None):
    model = pm.Model()
    with model:
    #with pm.Model() as model:
        #create string and associate with class of distributions - print to check 
        str_family = 'pm.glm.families.'+ dist_family + '()'
        dynamic_class = getattr(pm.glm.families,dist_family)
        family = dynamic_class()
        print(family)
        
        #if prior variable is true, use user-specified priors (need to be defined outside of function - also need
        #to define all priors in one go (for all variables that will be eventually used))
        if prior == True:
            pm.GLM.from_formula(formula,data,family=family,priors=my_priors)
        
        #if prior variable is false, use default priors
        else:
            pm.GLM.from_formula(formula,data,family=family)

        trace = pm.sample(draws=draws,tune=tune)
        #summary = pm.summary(trace)
        #traces = pm.traceplot(trace)
        #loo_array = pm.loo(trace,model,pointwise=True)
        #loo = pd.DataFrame(loo_array,index=['LOO','LOO_se','p_LOO','shape_warn','LOO_i'])
    #mod = model
    
    return trace, model #empty, traces,

In [ ]:
#6 - CHOOSE MODEL VALUES AND SET PRIORS

#Choose model values (dist_family = distribution family to sample likelihood/default priors from)
dist_family = 'StudentT' #'Normal'

#Set up observed data (equals dataframe)
data = df

#Set number of draws and number of burn-in (tune) samples
draws = 10000
tune = 5000

#Choose user-specified priors/hyperparameter priors - need same names as variables (e.g. 'Median_NDVI'/other 
#distribution - specific priors (e.g. 'lam' for StudentT)
#NB: if you set your own priors, you have to specify ones for all independent variables 
user_priors = {"Intercept": pm.Normal.dist(mu=0,sigma=100),
         "Median_NDVI": pm.Normal.dist(mu=0,sigma=100),
         "Mean_slope_deg": pm.Normal.dist(mu=50,sigma=50),
         "Mean_erod": pm.Normal.dist(mu=0,sigma=100),
         "sd": pm.HalfCauchy.dist(beta=2),
         "lam": pm.Normal.dist(mu=0,sigma=1),
         "hyper_param1": pm.HalfCauchy.dist(beta=1)
              }

In [ ]:
#7 - SET UP VARIABLE COMBINATIONS, WRITE PATSY FORMULAS, AND RUN MODEL
#Create variable combinations and run model on each (saves trace and model objects for all models run)
#Runs models for all combinations of variables (so 9 independent variables = 512 models total)
#NB: this takes a while to run (512 models without sampling for erosion uncertainty = ~1 full day)

#Get number of independent variables (params) and add 1 - will be used to give loop number of variables to include in
#var combinations (e.g. 1 = bivariate, 2 = trivariate, use params length so will include all bivariate up combos)
params = df.copy()
params = df.drop(columns=['Erosion_mmkyr'])
dflen = len(params.columns)
dflen_sum = np.sum([dflen,1])
a = np.arange(1,dflen_sum)

#Initialize trace (to store trace info from each model run) and trace number (kind, starts at 0 outside loop)
kind = 0
trace = {}
model = {}

#Start for loop
for i in a:
    
    #Create different combinations of independent variables (params) of length i (1 to total number of independent
    #variables, determined by a above)
    #pdb.set_trace()
    test = combinations(params,i)
    
    #Create Patsy-style strings (e.g. 'Erosion_mmkyr ~ Median_NDVI') to use in run_model function
    for k in list(test):
        
        #Starts count on trace number (first run = 0, second run = 1, etc., variables used stored in trace)
        #kind += kind
        kind = kind+1
        kind_str = "name"+str(kind)
        #print(kind_str)
        #Initial string creation (gives bad string like ('Erosion_mmkyr',+,'Median_NDVI') - so following formulas, e.g.
        #formula_2, just remove parts of string that pm.model will get mad at)
        formula_1 = 'Erosion_mmkyr ~ ' + ' + ' .join([str(k[0:len(k)])])
        formula_2 = formula_1.replace("(","")
        formula_3 = formula_2.replace("'","")
        formula_4 = formula_3.replace(")","")
        formula_5 = formula_4.replace(", "," + ")
        formula = formula_5.replace(",","")
        #print(formula)
        
        #name = str()
        
        #Run model - saves traces, models, and leave-one-out CV data as trace[1]/model[1], trace[2]/model[2], etc.
        trace[kind], model[kind] = run_model(formula,dist_family,data,draws,tune,prior=False,my_priors=None)
        #trace[kind] = run_model(formula,dist_family,data,draws,tune,prior=False,my_priors=None)

In [ ]:
#Initial summary analysis (model coefficients)

#Create summary tables of results of MCMC traces (includes mean value of parameter, standard deviation of parameters,
#MCMC standard error - divides trace into different groups and calculates mean/sd for each group, highest posterior
#density (2.5 - 97.5%)/credibility interval, number of effective samples (# of samples that are effectively independent),
#and Gelman-Rubin statistic (see above, < 1.1 is good but not perfect diagnostic))
#NB: tables look bad, but not going to bother creating multi-index df here 

#Create empty dictionaries/dataframe to store data
trace_sum = {}

for i in trace:
    trace_sum[i] = pm.summary(trace[i])
    trace_sum[i].loc[4] = i
    trace_sum[i].rename(index = {4: "Model"},inplace=True)
    trace_sum[i] = trace_sum[i].transpose()
    print(trace_sum[i])

In [ ]:
#Check autocorrelation plots - if there is any sort of coherent pattern (doesn't just hover at approx. 0) it's auto-
#correlated and will need more draws/larger burn-in

#NB: Each variable (intercept, lambda for student t dist, independent variables) will have an autocorrlation plot for
#each chain (e.g. 4 chains = four intercept plots per model run (0-3))

#Change max_lag to plot bigger/smaller lag
for i in trace:
    autocorr_plot = pm.autocorrplot(trace[i],max_lag = 1000)
    #figtitle = 'Model #' +str(i)
    #print(figtitle)

In [ ]:
#Check traceplots - chains should overlap a lot in both the posterior distributions for variables (right graphs) and
#on the traceplots themselves (left graphs) - if one chain (represented by one line) is far away from others, will
#need to rerun model with a larger burn-in and more total draws
x = len(trace)
for i in trace:
    trc_plot = pm.traceplot(trace[i])

In [ ]:
#9 - MODEL COMPARISON
#Prior to individual model checks, compare models using information criterion/PSIS-LOO (Vehtari et al., 2017) to only
#keep the best models (otherwise will end up with a lot for doing posterior checks, etc.), compares out-of-sample
#prediction error between models (computes expected log predictive densities) and correcting for number of parameters

#PSIS-LOO comparison (Pareto smoothed importance sampling leave-one-out cross validation) - another method of model
#comparison (Vehtari et al., 2017) - LOO trains the model on n-1 samples and tests against the one holdout sample and
#does this n number of times, and estimates the out-of-sample predictive accuracy each time and can help assess over/
#underfitting, PSIS methods mean that computation is more efficient 

#k-hat - diagnostic for PSIS-LOO, if the value is greater than 0.7 for any pointwise estimates, it means that observation
#is very influential and model needs to be reparameterized (PyMC3 will throw up an error if that occurs, otherwise
#k-hat calculations are all backend)

#NB: To run WAIC (Widely Applicable Information Criterion), need to change info below

#Create empty dictionary and dataframe (populated in for loop)
kind = 0
psisloo = {}
total_loo = []

#loop creates dictionary of model PSIS-LOO values (e.g. psisloo[1] = values for model 1) and appends them to a dataframe
for i in trace:
    kind = kind+1
    #title = 'WAIC_Model #' +str(i)
    psisloo[kind] = pm.loo(trace[i],model[i])
    #column_name = 'LOO_Model #' +str(i)
    column_name = i
    loo_index = ['LOO','LOO_se','p_LOO','shape_warn']
    loo_df = pd.DataFrame(psisloo[kind],index=loo_index,columns=[column_name])
    total_loo.append(loo_df)
    #print(total_loo)

#Clean-up resulting dataframe
total_loo = pd.concat(total_loo,axis=1)

#Visualizing LOO values for each model (lower is better)
    
#Combine all LOO values (dataframes from for loop) into single dataframe, columns are different models, index = LOO vals, 
#create a model row (contains model name)
total_loo.loc[4]=total_loo.columns
total_loo.rename(index ={4:"Model"},inplace=True)
test_loo = total_loo.transpose() #DO NOT CHANGE THE NAME OF TEST_LOO

In [ ]:
#Plot PSIS-LOO data and standard errors

#Create x, y, and y-error variables for plotting LOO values
x = test_loo["Model"]
y = test_loo["LOO"]
y_err = test_loo["LOO_se"]

#Plot data
fig, ax = plt.subplots(figsize=(50,10))
ax.errorbar(x,y,yerr=y_err,fmt='-o')
ax.set_xlabel('Model')
ax.set_ylabel('LOO')
ax.set_title('LOO values and errors')
plt.xticks(rotation=45)

In [ ]:
#WAIC (Widely-Applicable Information Criterion, Watanabe, 2010) - improvement of deviance information criterion (DIC) 
#tends to fail very influential observations (model with observation is very different than model without observation, 
#lpd > 0.4) or with really weak priors - PyMC3 shows error in cases where lpd > 0.4

#NB: To run, will need to remove all # below

#Create empty dictionary and dataframe (populated in for loop)
#kind = 0
#waic = {}
#total_waic = []

#loop creates dictionary of model WAIC values (e.g. waic[1] = WAIC values for model 1) and appends them to a dataframe
#for i in trace:
#    kind = kind+1
    #title = 'WAIC_Model #' +str(i)
#    waic[kind] = pm.waic(trace[i],model[i])
#    column_name = i
#    waic_index = ['WAIC','WAIC_se','p_WAIC','var_warn']
#    waic_df = pd.DataFrame(waic[kind],index=waic_index,columns=[column_name])
#    total_waic.append(waic_df)
    #print(total_waic)

#Clean-up resulting dataframe
#total_waic = pd.concat(total_waic,axis=1)

#Visualizing WAIC values for each model (lower is better)
    
#Combine all WAIC values (dataframes from for loop) into single dataframe, columns are different models, index = WAIC 
#vals, create a model row (contains model name)
#total_waic.loc[4]=total_waic.columns
#total_waic.rename(index ={4:"Model"},inplace=True)
#test_waic = total_waic.transpose()

In [ ]:
#Function to delete models/traces based on LOO/WAIC values and a user-defined cutoff (e.g. top 5 models), results in 
#new dictionaries with the top models/traces

#NB: can skip this part and just access models you're interested in by hand (particuarly because none of the bivariate
#models made it into the top 50 or so models (lowest PSIS-LOO values)), just set cutoff as total number of models (e.g. 512)

#NB: do need to have a dictionary called 'top_traces' to run a function below ('predict_erosion' and 'model_tester'),
#so if you want all models, just set cutoff as total number of models initially run above

#User-defined variables = cutoff (number of models to save) and method (default is LOO, can choose 'WAIC' also)
def remove_models(cutoff,method=None):  
    
    #Create empty dictionaries for top models and corresponding traces
    top_mod = {}
    top_trace = {}
    
    #if method is specified as WAIC, use test_WAIC values
    if method == 'WAIC':
        #Create column in test_waic dataframe of sort order (1 = lowest WAIC value)
        test_waic.loc[:,'sortorder'] = test_waic.WAIC.rank()
        for i in test_waic.index:
            if test_waic.sortorder[i] <= cutoff:
                top_mod[i] = model[i]
                top_trace[i] = trace[i]
    
    #default method is LOO
    else:
        #Create column in test_waic dataframe of sort order (1 = lowest LOO value)
        test_loo.loc[:,'sortorder'] = test_loo.LOO.rank() 
        for i in test_loo.index:
            if test_loo.sortorder[i] <= cutoff:
                top_mod[i] = model[i]
                top_trace[i] = trace[i]
    
    #Results in top models/traces based on user-defined cutoff
    return top_mod, top_trace

In [ ]:
#Run remove_models function above - don't change name of 'top_models' or 'top_traces'
#NB: Will not remove models with collinear variables (e.g. slope, local relief) - will need to delete those by hand

top_models, top_traces = remove_models(10)

In [ ]:
#Use the following to set top_traces and top_models by hand, just change numbers to which models you are interested in
#(to see what vars are in what models, can check by running model[#], shows you what vars are there)

#Set top models and traces by hand - above code does not remove collinear data, so some collinear models potentially 
#included in top models

#top_models = {178: model[178], 322: model[322], 288: model[288], 67: model[67], 413: model[413],
#             323: model[323], 181: model[181], 283: model[283], 180: model[180], 437: model[437]}
#top_traces = {178: trace[178], 322: trace[322], 288: trace[288], 67: trace[67], 413: trace[413],
#             323: trace[323], 181: trace[181], 283: trace[283], 180: trace[180], 437: trace[437]}

In [ ]:
#Get keys from top_models (and top_traces) dictionaries to check
model_nums = list(top_models.keys())
model_nums

In [ ]:
#10 - INDIVIDUAL MODEL ASSESSMENT
#After only the best models are left, need to check how good each model is individually + assess model coefficeints of
#different parameters, certainity around each parameter, etc.

#Create summary tables of results of MCMC traces (includes mean value of parameter, standard deviation of parameters,
#MCMC standard error - divides trace into different groups and calculates mean/sd for each group, highest posterior
#density (2.5 - 97.5%)/credibility interval, number of effective samples (# of samples that are effectively independent),
#and Gelman-Rubin statistic (see above, < 1.1 is good but not perfect diagnostic))
#NB: tables look bad, but not going to bother creating multi-index df here 

#Create empty dictionaries/dataframe to store data
trace_sum = {}

for i in top_traces:
    trace_sum[i] = pm.summary(trace[i])
    trace_sum[i].loc[4] = i
    trace_sum[i].rename(index = {4: "Model"},inplace=True)
    print(trace_sum[i])

#To access individual trace_sum dataframes - change number to access different ones
trace_sum[1]

In [ ]:
#Create forest plot of all variables from all models (compare means/CI for model params between models and chains)

#Create tuples of top traces and trace names for input into forest plot function (k = key, v = value)
trace_tuple = [(v) for k,v in top_traces.items()]
trace_name_tuple = [(k) for k,v in top_traces.items()]

#Create forest plot - can set to plot individual/fewer traces by using [trace[1],trace[7], etc.] instead of trace_tuple,
#can show individual chains (4 per model with default values) by removing combined=True, can also choose what vars to 
#plot by adding var_names='Intercept', etc. (needs to be exact variable name), more options see PyMC3 documentation
#NB: CHECK MODEL NAMES! Do the models have the correct names (e.g. model with most vars below = actual model with most)
forest_plot = pm.forestplot(trace_tuple, model_names = trace_name_tuple, combined=True, figsize=(10,25))



In [ ]:
#Plot posterior distribution of parameters (independent variables, intercept, and vars associated with data likelihood
#distribution), show mean values for each parameter and 95% credibility intervals

#Create list of variables to make posterior distributions for (independent vars from df, intercept, and parameter ass-
#ciated with likelihood distribution - e.g. lambda for student t)
ind_var = df.drop(columns=['Erosion_mmkyr'])
ind_var = ind_var.columns
posterior = ind_var.insert(0,"Intercept")
posterior_params = posterior.insert(0,"lam")

#Create range of practical equivalence (ROPE) - 0.1/-0.1*sd of y (erosion rate data)
y_std = df.iloc[:,0].std
rope_vals = (round((y_std()*0.1),2),round((y_std()*-0.1),2))

#Plot posterior - it doesn't like to display more than one trace at a time, so need to edit it by hand (e.g. top_traces
#[6], top_traces[1], etc.) - need to specify which trace you want to do!
test = pm.plot_posterior(top_traces[1],var_names = posterior_params,figsize=(15,5), credible_interval=0.95,
                        rope=rope_vals)


In [ ]:
#Posterior Predictive Checks (PPC)
#PPC generate new observations from your model and compare the distributions of new observations to the distribution of
#the true observed data - looking for where the two distributions deviate from each other (indicates location where
#model is failing and should be corrected)
#NB: not definitive check and not for model comparison - runs into problem of using data twice (once to fit the model
#and another time to check the predictions of the model), but can provide broad overview (see Gelman et al., 2004)
np.set_printoptions(threshold=np.inf)

ppc = pm.sample_posterior_predictive(top_traces[67], samples = 1000, model = top_models[67])
x = np.asarray(ppc['y']).shape
ppc_val = np.array(list(ppc.values()))
ppc_val

In [ ]:
#Posterior predictive checks (PPCs, see Gelman et al 2004 for more details)
#Test mean - calculates erosion rates (generating data from posterior distributions of model params), 
#calculate fraction of values that are more extreme than the true erosion rate mean
#p-values close to 0.5 are good - means model is not systematically over/underpredicting data!

In [ ]:
#Caclulates p-value for mean test statistic ()

#Samples from user-set model (need to specify trace also) posterior probability distributions (model params), and 
#calculates erosion rates with given independent variables, then calculates the mean erosion rate from generated data
def test_statistic_mean(trace, model, num_samples, num_sims):
    #total_sims = len(num_sims)
    erosion = df.iloc[:,0]
    true_mean = erosion.mean()
    sums = 0
    for i in range(num_sims):
        ppc_n = pm.sample_posterior_predictive(trace, samples = num_samples, model = model)
        ppc_np = np.array(list(ppc_n.values()))
        ppc_mean = ppc_np.mean()
        if ppc_mean > true_mean:
            sums += 1
        else:
            sums += 0
    total_sum = (1/num_sims)*sums
    return total_sum

In [ ]:
#Run test_statistic_mean and print
test_mean = test_statistic_mean(trace[1],model[1],1000,500)
test_mean

In [ ]:
#Calculate erosion rates using the 95% credible intervals (2.5 and 97.5) and mean predicted erosion rate to see range
#of values resulting from uncertainty in model parameters

#Specify trace, model (should be the same), and number of samples you want it to draw from posterior prob distributions
def mean_hpd_calc(trace, model, num_samples):
    ppc_mean_hpd = pm.sample_posterior_predictive(trace,num_samples, model = model, progressbar=False)
    ppc_np = np.asarray(ppc_mean_hpd['y'])
    ppc_df = pd.DataFrame(ppc_np)
    ppc_df = ppc_df.transpose()
    
    #Create empty dataframe for storing means, HPD
    res = pd.DataFrame(index = range(0,len(ppc_df)))
    
    #add true erosion values to the dataframe
    res.loc[:,'True_erosion'] = np.array(df.iloc[:,0])
    
    #Populate results dataframe with means and 95% (2.5, 97.5) credible interval data
    for i in range(0,len(ppc_df)):
        res.loc[i,'Estimated_mean'] = np.mean(ppc_df.loc[i,:])
        res.loc[i,'HPD_2.5'] = pm.stats.hpd(ppc_df.iloc[i,:])[0]
        res.loc[i,'HPD_97.5'] = pm.stats.hpd(ppc_df.iloc[i,:])[1]
        
        
    return res
        
    

In [ ]:
#Plot mean values and 95% HPD (CI) for given model, to see different models, need to change trace/model number by hand
ppc_predict = mean_hpd_calc(trace[1], model[1], 1)

#Plot data to check it out
fig, ax = plt.subplots()

#have to change y axis name to access the 95% CI 
sns.regplot(x='True_erosion', y='Estimated_mean', data=ppc_predict, ax=ax)
#sns.regplot(x='True_erosion', y='HPD_2.5', data=ppc_predict, ax=ax)
#sns.regplot(x='True_erosion', y='HPD_97.5', data=ppc_predict, ax=ax)
plt.ylim(-3,15)
plt.show()


In [ ]:
#Comparing true and predicted erosion rates (using mean model parameter values and sampling from posterior prob dist-
#ributions) - sampling from posterior function is below


#Calcuate expected erosion rate given mean model parameters and indepedent variables
#Specify which model (only need to define trace number) and input the dataframe containing independent variables
def predict_erosion_mean(trace,df):
    trace_sum = pm.summary(trace)
    mean_params = trace_sum.iloc[:,0]
    #mean_params = pd.DataFrame(mean_params)
    #mean_params = np.array(mean_params)
    
    ind_vars = df.drop(columns=["Erosion_mmkyr"])
    erosion = df.iloc[:,0]
    
    predict_oneval = pd.DataFrame(index = ind_vars.index)
    predict_oneval.loc[:,'Predicted_Erosion'] = 0
    #pdb.set_trace()
    for i in ind_vars:
        try:
            predict_oneval.loc[:,'Predicted_Erosion'] += (mean_params[i]*ind_vars[i])
        #if ind_vars contains variables not in the trace (ind_vars contains all independent variables, there will be
        #a KeyError - code will skip that column/value if it gets a KeyError
        except KeyError:
            pass
    predict_oneval.loc[:,'Predicted_Erosion'] += mean_params[0]
    return predict_oneval
    

In [ ]:
#Run predict_erosion_mean to get predicted erosion rates with mean (highest probability) model params 

#Specify which model/trace you want to use, will need to change by hand to see other models
outs_mean = predict_erosion_mean(trace[1],df)

#Define which model you set as the trace to test, don't need to define really (can leave as Model_X), but need to 
#remember which model/trace is actually being run
model_name = 'Model_X'
#model_name = 'Model_67'

#Concatenate two dataframes by columns
mean_erosion_concat = pd.concat([df.iloc[:,0],outs_mean],axis=1)

#Rename 'Predicted_Erosion' column with the name 'Model_' based on trace sampled
mean_erosion_concat.rename(columns={"Predicted_Erosion": model_name}, inplace=True)

In [ ]:
#Calculate R^2 and MAE comparing predicted erosion rates (sampling from parameters) to true erosion rates
r2 = r2_score(mean_erosion_concat.Erosion_mmkyr,mean_erosion_concat.Model_X)
mae = mean_absolute_error(mean_erosion_concat.Erosion_mmkyr,mean_erosion_concat.Model_X)

r2

In [ ]:
#Calcuate expected erosion rate while sampling from posterior prob distributions for model parameters and 
#given the measured indepedent variables

#Specify model/trace (trace only) you want to sample from, dataframe with independent variables, and the number of samples to draw
#from model param posterior probability distributions
def predict_erosion(trace,df,num_samples):
    num_obs = len(df.index)
    params = np.random.choice(trace,size=num_samples,replace=False)
    
    #Put parameters into a dataframe with the number of elements equal to num_samples.
    #For some reason this ended up with a duplicate index? doesn't affect anything else
    param_test = {}
    for i in range(len(params)):
        param_test[i] = pd.DataFrame(params[i],index = [i])
    params = pd.concat(param_test)
    ind_vars = pd.DataFrame(index = range(num_obs*num_samples),columns = df.columns)
    for j in df.index:
        ind_vars.loc[j*num_samples:(j+1)*num_samples-1,:]= np.array(df.loc[j,:])
    
    #define number of observations
    predict = pd.DataFrame(np.zeros([num_obs*num_samples,0])) #Dataframe size is # samples * # observations
    modname = 'Model_'+str(trace)
    #pdb.set_trace()
    predict.loc[:,'Predicted_Erosion'] = 0
    #predict.loc[:,modname] = 0
    for i in ind_vars:
        if i is 'Erosion_mmkyr':
            pass
        else:
            try:
                predict.loc[:,'Predicted_Erosion'] += np.tile(params.loc[:,i].values,num_obs)*ind_vars[i].values
            #if ind_vars contains variables not in the trace (ind_vars contains all independent variables, there will be
            #a KeyError - code will skip that column/value if it gets a KeyError
            except KeyError:
                pass
    #pdb.set_trace()

    predict.loc[:,'Predicted_Erosion'] += np.tile(params.loc[:,'Intercept'].values,num_obs)
    
    return predict#, model_name

In [ ]:
#Run predict_erosion function with trace/number of samples (e.g. num_samples = 10, 0-9 are predicted rates for the
#first catchment, 10-19 for the second catchment, etc.)
num_samples = 1000
outs = predict_erosion(top_traces[1],df,num_samples)


In [ ]:
#Create dataframe of true erosion rates equal to the size of the predicted erosion rates above (e.g. 1000 samples for
#predicted erosion rates above = number of true erosion rates * 1000 )
#NB: calls on 'top_traces' from above (where some models are removed based on PSIS-LOO or WAIC values, if the cutoff
#number of models is set to all of them (e.g. 512), this will take a while to run)


modpred = pd.DataFrame(df.iloc[:,0])
erosion = df.iloc[:,0]
model_tester = pd.DataFrame(np.zeros([num_samples*len(df.iloc[:,0]),0]))

for traces in top_traces:
    modname = 'Model_'+str(traces)
    modpred.loc[:,modname] = predict_erosion(top_traces[traces],df,num_samples)
for i in erosion.index:
    #pdb.set_trace()
    model_tester.loc[i*num_samples:(i+1)*num_samples-1,'erosion_measured'] = df.iloc[:,0][i]
    for traces in top_traces:
        modname = 'Model_'+str(traces)
        model_tester.loc[i*num_samples:(i+1)*num_samples-1,modname] = modpred.loc[i,modname]

model_tester

In [ ]:
#Combine true erosion rates (model_tester) and the predicted erosion rates from predict_erosion function (outs)

#Define which model you set as the trace to test, will need to change by hand unforunately
model_name = 'Model_1'

#Concatenate two dataframes by columns
erosion_concat = pd.concat([model_tester,outs],axis=1)

#Drop the column with name 'Model _' - based on trace sampled
combined_erosion = erosion_concat.drop(model_name,axis=1)
#combined_erosion = erosion_concat.drop(modname,axis=1)

#Rename 'Predicted_Erosion' column with the name 'Mode;_' based on trace sampled
combined_erosion.rename(columns={"Predicted_Erosion": model_name}, inplace=True)
#combined_erosion.rename(columns={"Predicted_Erosion": modname}, inplace=True)

#Check combined erosion dataframe and make sure that erosion_measured still okay and that predicted_erosion has been 
#replaced by model_ based on the trace sampled
combined_erosion


In [ ]:
#Calculate statistics comparing predicted erosion rates (sampling from parameters above) to true erosion rates
#**for all, will need to change combined_erosion.Model_X to whichever model you're using
r2 = r2_score(combined_erosion.erosion_measured,combined_erosion.Model_1)
mae = mean_absolute_error(combined_erosion.erosion_measured,combined_erosion.Model_1)

r2


In [ ]:
#11 - TROUBLESHOOTING
#Following code is for troubleshooting Bayesian modelling part, e.g. issues with acceptance rate, divergences, etc.

In [ ]:
#If original model runs through up a divergence error (e.g. '1 divergernce after tuning'), can test where in prob. 
#space divergences seem to be occurring (if clustered, suggests that there's a region of probability space that
#the MCMC sampler is not exploring well, need to increase tuning size (burn-in) or reparameterize)

#See Gabry et al 2019 (https://rss.onlinelibrary.wiley.com/doi/full/10.1111/rssa.12378) for more info

#Following code from: https://docs.pymc.io/notebooks/Diagnosing_biased_Inference_with_Divergences.html
def pairplot_divergence(trace, ax=None, divergence=True, color='C3', divergence_color='C2'):
    theta = trace.get_values(varname='theta', combine=True)[:, 0]
    logtau = trace.get_values(varname='tau_log__', combine=True)
    if not ax:
        _, ax = plt.subplots(1, 1, figsize=(10, 5))
    ax.plot(theta, logtau, 'o', color=color, alpha=.5)
    if divergence:
        divergent = trace['diverging']
        ax.plot(theta[divergent], logtau[divergent], 'o', color=divergence_color)
    ax.set_xlabel('theta[0]')
    ax.set_ylabel('log(tau)')
    ax.set_title('scatter plot between log(tau) and theta[0]');
    return ax

#Run above function and plot
pairplot_divergence(trace[x]);

In [ ]:
#Check sampler statistics (code set up to use default No-U-Turn sampler)

#Have to check each model/trace sampler statistics individually, just change value in [] to see different traces

#Get list of statistics associated with sampler available to check
trace[x].stat_names

#E.g., check the mean acceptance rate, just need to change trace name
trace[x].get_sampler_stats('mean_tree_accept')